In [3]:
import os
import csv
import json
import pandas as pd
import natsort
import datetime

In [99]:
def heart_file(fname,count,uname):
    f = open('sokulee/' + uname + '/' + fname) # file open

    json_test = json.loads(f.read())

    emp_data = json_test['activities-heart-intraday']
    # test = json_test['activities-heart']
    filename = 'all_user_hearts.csv'
    employ_data = open(filename, 'a')
    csvwriter = csv.writer(employ_data)
    li = emp_data['dataset']

    for emp in li:
        emp['USERNAME'] = uname
        emp['DATE'] = json_test['activities-heart'][0]['dateTime']
        if count == 0:
            header = emp.keys()
            csvwriter.writerow(header)
            count = 1
        csvwriter.writerow(emp.values())
    employ_data.close()

In [1]:
## data가 없는 파일은 제외 시켰음(A08_20160401, A052_20160402, A073_20160402)

def steps_file(fname,step_count,uname):
    f = open('sokulee/' + uname + '/' + fname) # file open

    json_test = json.loads(f.read())
    
    emp_data = json_test['activities-steps-intraday']
    # test = json_test['activities-heart']
    filename = 'all_user_steps.csv'
    employ_data = open(filename, 'a')
    csvwriter = csv.writer(employ_data)
    li = emp_data['dataset']

    for emp in li:
        emp['USERNAME'] = uname
        emp['DATE'] = json_test['activities-steps'][0]['dateTime']
        if step_count == 0:
            header = emp.keys()
            csvwriter.writerow(header)
            step_count = 1
        csvwriter.writerow(emp.values())
    employ_data.close()

In [2]:
def sleeps_file(fname,sleep_count,uname):
    f = open('sokulee/' + uname + '/' + fname) # file open

    json_test = json.loads(f.read())
    if bool(json_test['sleep']) == False:
        return -1
    emp_data = json_test['sleep'][0]

    filename = 'all_user_sleeps.csv'
    employ_data = open(filename, 'a')
    csvwriter = csv.writer(employ_data)
    # li = emp_data['dataset']
    emp = {}
    emp['USERNAME'] = uname
    emp['DATE'] = json_test['sleep'][0]['dateOfSleep']
    emp['STARTTIME'] = json_test['sleep'][0]['minuteData'][0]['dateTime']
    emp['MINUTESASLEEP'] = json_test['sleep'][0]['minutesAsleep']
    emp['MINUTESAWAKE'] = json_test['sleep'][0]['minutesAwake']
    emp['TIMEINBED'] = json_test['sleep'][0]['timeInBed']
    
    if sleep_count == 0:
        header = emp.keys()
        csvwriter.writerow(header)
        sleep_count = 1
    csvwriter.writerow(emp.values())
    employ_data.close()

In [6]:
user_list = os.listdir('sokulee/')
user_list = natsort.natsorted(user_list)
user_list.remove('.DS_Store')

In [7]:
count = 0
step_count = 0
sleep_count = 0
for i in user_list:
    ul = os.listdir('sokulee/' + i + '/')
    heart_list = []
    steps_list = []
    sleep_list = []
    ul.sort()
    
    for j in ul:
        if 'heart' in j:
            heart_list.append(j)
        if 'steps' in j:
            steps_list.append(j)
        if 'sleep' in j:
            sleep_list.append(j)
            
    for heart_index in range(len(heart_list)):
        heart_file(heart_list[heart_index],count,i)
        count = 1
    for step_index in range(len(steps_list)):
        steps_file(steps_list[step_index],step_count,i)
        step_count = 1
        
    for sleep_index in range(len(sleep_list)):
        sleeps_file(sleep_list[sleep_index],sleep_count,i)
        sleep_count = 1
        print(sleep_list[sleep_index], ' ', i)


In [47]:
all_heart = pd.read_csv('all_user_hearts.csv')
all_heart = pd.DataFrame(all_heart, columns=['USERNAME','DATE','time','value'])
all_heart = all_heart.rename(columns = {'value': 'VALUE'})
all_heart = all_heart.rename(columns = {'time': 'TIME'})

In [49]:
all_heart = all_heart.to_csv('all_user_hearts.csv',index=False)

In [77]:
all_steps = pd.read_csv('all_user_steps.csv')
all_steps.head(8)

,TIME,STEPS,USERNAME,DATE
0,00:00:00,0,A01,2016-04-01
1,00:01:00,0,A01,2016-04-01
2,00:02:00,0,A01,2016-04-01
3,00:03:00,0,A01,2016-04-01
4,00:04:00,0,A01,2016-04-01
5,00:05:00,0,A01,2016-04-01
6,00:06:00,16,A01,2016-04-01
7,00:07:00,0,A01,2016-04-01


In [78]:
date = datetime.datetime.strptime(all_steps.iloc[1]['TIME'],"%H:%M:%S")

In [105]:
all_steps = all_steps[all_steps.TIME != 'TIME']

In [115]:
all_steps['HOUR'] = all_steps['TIME'].apply(lambda x : pd.to_datetime(x).hour )

/home/khz/anaconda3/envs/nack/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [120]:
all_steps['MINUTE'] = all_steps['TIME'].apply(lambda x : pd.to_datetime(x).minute )

/home/khz/anaconda3/envs/nack/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [123]:
all_steps = pd.DataFrame(all_steps, columns=['USERNAME','DATE','TIME','HOUR','MINUTE','STEPS'])

In [125]:
all_steps = all_steps.to_csv('all_user_steps.csv',index=False)

In [4]:
all_sleeps = pd.read_csv('all_user_sleeps.csv')
all_sleeps.head(5)

,USERNAME,DATE,STARTTIME,MINUTESASLEEP,MINUTESAWAKE,TIMEINBED
0,A01,2016-04-01,02:39:00,485,26,511
1,A01,2016-04-02,02:09:00,512,37,549
2,A01,2016-04-04,01:52:30,335,11,346
3,A01,2016-04-05,02:04:00,311,24,335
4,A01,2016-04-06,01:49:00,491,59,551
